In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/customer-support-ticket-dataset/customer_support_tickets.csv


# 1. Context

I'm doing this as a practical analysis project. All data proviced is public domain & via Kaggle's internal dataset - [Source Dataset: Customer Support Ticket Dataset](https://www.kaggle.com/code/aniketg11/support-tickets-classification/data). 

![

We'll be analyzing some relevant support trend metrics with this dataset, including:

    - Ticket Type
    - Ticket Subject
    - Resolution Time
    - Priority
    - First Response Time
    - Time to Resolution
    - Ticket Channel
    - Customer Satisfaction

In [2]:
import pandas as pd
import numpy as np
import matplotlib as plt

In [3]:
cs_tickets = pd.read_csv("/kaggle/input/customer-support-ticket-dataset/customer_support_tickets.csv")

# 2. Data Preview

Get a look at the data I'm working with to understand shape, data types, null/dupe count. 

## 2.1 Null/Dupe Count

In [4]:
cs_tickets.head()

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,Ticket ID,Customer Name,Customer Email,Customer Age,Customer Gender,Product Purchased,Date of Purchase,Ticket Type,Ticket Subject,Ticket Description,Ticket Status,Resolution,Ticket Priority,Ticket Channel,First Response Time,Time to Resolution,Customer Satisfaction Rating
0,1,Marisa Obrien,carrollallison@example.com,32,Other,GoPro Hero,2021-03-22,Technical issue,Product setup,I'm having an issue with the {product_purchase...,Pending Customer Response,NaN,Critical,Social media,2023-06-01 12:15:36,NaN,NaN
1,2,Jessica Rios,clarkeashley@example.com,42,Female,LG Smart TV,2021-05-22,Technical issue,Peripheral compatibility,I'm having an issue with the {product_purchase...,Pending Customer Response,NaN,Critical,Chat,2023-06-01 16:45:38,NaN,NaN
2,3,Christopher Robbins,gonzalestracy@example.com,48,Other,Dell XPS,2020-07-14,Technical issue,Network problem,I'm facing a problem with my {product_purchase...,Closed,Case maybe show recently my computer follow.,Low,Social media,2023-06-01 11:14:38,2023-06-01 18:05:38,3.0
3,4,Christina Dillon,bradleyolson@example.org,27,Female,Microsoft Office,2020-11-13,Billing inquiry,Account access,I'm having an issue with the {product_purchase...,Closed,Try capital clearly never color toward story.,Low,Social media,2023-06-01 07:29:40,2023-06-01 01:57:40,3.0
4,5,Alexander Carroll,bradleymark@example.com,67,Female,Autodesk AutoCAD,2020-02-04,Billing inquiry,Data loss,I'm having an issue with the {product_purchase...,Closed,West decision evidence bit.,Low,Email,2023-06-01 00:12:42,2023-06-01 19:53:42,1.0


In [5]:
cs_tickets.shape
cs_tickets.info()
cs_tickets.isna().mean().sort_values(ascending=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8469 entries, 0 to 8468
Data columns (total 17 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Ticket ID                     8469 non-null   int64  
 1   Customer Name                 8469 non-null   object 
 2   Customer Email                8469 non-null   object 
 3   Customer Age                  8469 non-null   int64  
 4   Customer Gender               8469 non-null   object 
 5   Product Purchased             8469 non-null   object 
 6   Date of Purchase              8469 non-null   object 
 7   Ticket Type                   8469 non-null   object 
 8   Ticket Subject                8469 non-null   object 
 9   Ticket Description            8469 non-null   object 
 10  Ticket Status                 8469 non-null   object 
 11  Resolution                    2769 non-null   object 
 12  Ticket Priority               8469 non-null   object 
 13  Tic

Customer Satisfaction Rating    0.673043
Time to Resolution              0.673043
Resolution                      0.673043
First Response Time             0.332861
Ticket Description              0.000000
Ticket Channel                  0.000000
Ticket Priority                 0.000000
Ticket Status                   0.000000
Ticket ID                       0.000000
Customer Name                   0.000000
Ticket Type                     0.000000
Date of Purchase                0.000000
Product Purchased               0.000000
Customer Gender                 0.000000
Customer Age                    0.000000
Customer Email                  0.000000
Ticket Subject                  0.000000
dtype: float64

In [6]:
dupe_count = cs_tickets.duplicated().sum()
print(f"Number of Duplicate Rows: {dupe_count}")

Number of Duplicate Rows: 0


In [7]:
missing_count = cs_tickets.isna().sum().sort_values(ascending=False)
print("Missing Values per Column:")
print(missing_count)

Missing Values per Column:
Customer Satisfaction Rating    5700
Time to Resolution              5700
Resolution                      5700
First Response Time             2819
Ticket Description                 0
Ticket Channel                     0
Ticket Priority                    0
Ticket Status                      0
Ticket ID                          0
Customer Name                      0
Ticket Type                        0
Date of Purchase                   0
Product Purchased                  0
Customer Gender                    0
Customer Age                       0
Customer Email                     0
Ticket Subject                     0
dtype: int64


## 2.2 Cleaning

In [8]:
# Standardize categorical fields. 

cat_cols = ["Ticket Status", "Ticket Priority", "Ticket Channel", "Ticket Type"]

for col in cat_cols: 
        # Convert to string (in case there are any numbers or NaN), strip spaces, and lowercase
        cs_tickets[col] = (
            cs_tickets[col]
            .astype("string")
            .str.strip()    # remove leading/trailing spaces
            .str.lower()    # standardize to lowercase
        )

In [9]:
# Quick post cleaning check. 

print("Final shape:", cs_tickets.shape)
print("\nData types after cleaning:")
print(cs_tickets.dtypes)

print("\nSample of cleaned data:")
cs_tickets.head()

Final shape: (8469, 17)

Data types after cleaning:
Ticket ID                                int64
Customer Name                           object
Customer Email                          object
Customer Age                             int64
Customer Gender                         object
Product Purchased                       object
Date of Purchase                        object
Ticket Type                     string[python]
Ticket Subject                          object
Ticket Description                      object
Ticket Status                   string[python]
Resolution                              object
Ticket Priority                 string[python]
Ticket Channel                  string[python]
First Response Time                     object
Time to Resolution                      object
Customer Satisfaction Rating           float64
dtype: object

Sample of cleaned data:


/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,Ticket ID,Customer Name,Customer Email,Customer Age,Customer Gender,Product Purchased,Date of Purchase,Ticket Type,Ticket Subject,Ticket Description,Ticket Status,Resolution,Ticket Priority,Ticket Channel,First Response Time,Time to Resolution,Customer Satisfaction Rating
0,1,Marisa Obrien,carrollallison@example.com,32,Other,GoPro Hero,2021-03-22,technical issue,Product setup,I'm having an issue with the {product_purchase...,pending customer response,NaN,critical,social media,2023-06-01 12:15:36,NaN,NaN
1,2,Jessica Rios,clarkeashley@example.com,42,Female,LG Smart TV,2021-05-22,technical issue,Peripheral compatibility,I'm having an issue with the {product_purchase...,pending customer response,NaN,critical,chat,2023-06-01 16:45:38,NaN,NaN
2,3,Christopher Robbins,gonzalestracy@example.com,48,Other,Dell XPS,2020-07-14,technical issue,Network problem,I'm facing a problem with my {product_purchase...,closed,Case maybe show recently my computer follow.,low,social media,2023-06-01 11:14:38,2023-06-01 18:05:38,3.0
3,4,Christina Dillon,bradleyolson@example.org,27,Female,Microsoft Office,2020-11-13,billing inquiry,Account access,I'm having an issue with the {product_purchase...,closed,Try capital clearly never color toward story.,low,social media,2023-06-01 07:29:40,2023-06-01 01:57:40,3.0
4,5,Alexander Carroll,bradleymark@example.com,67,Female,Autodesk AutoCAD,2020-02-04,billing inquiry,Data loss,I'm having an issue with the {product_purchase...,closed,West decision evidence bit.,low,email,2023-06-01 00:12:42,2023-06-01 19:53:42,1.0


## 2.3 Fix Date Columns

In [10]:
date_cols = ["Date of Purchase", "First Response Time", "Time to Resolution"]

for col in date_cols:
    if col in cs_tickets.columns:
        cs_tickets[col] = pd.to_datetime(cs_tickets[col], errors="coerce")

# Verifying changes

cs_tickets[date_cols].isna().mean()

Date of Purchase       0.000000
First Response Time    0.332861
Time to Resolution     0.673043
dtype: float64

In [11]:
date_like_cols = [col for col in cs_tickets.columns if "date" in col.lower() or "time" in col.lower()]

In [12]:
print("Likely date columns:", date_like_cols)

Likely date columns: ['Date of Purchase', 'First Response Time', 'Time to Resolution']


In [13]:
cs_tickets = cs_tickets.copy()

datetime_cols = ["First Response Time", "Time to Resolution", "Date of Purchase"] 

for col in datetime_cols: 
        cs_tickets[col] = pd.to_datetime(cs_tickets[col], errors="coerce")

cs_tickets[datetime_cols].dtypes

First Response Time    datetime64[ns]
Time to Resolution     datetime64[ns]
Date of Purchase       datetime64[ns]
dtype: object

# 3. Analysis

Time to get some numbers. I'll now focus on getting metrics for the aforementioned 

In [14]:
# Establish the KPI metric for resolution time in hrs & mins. 

# Work with a copy of the cleaned data. 

cs_tickets["resolution_hours"] = (
    cs_tickets["Time to Resolution"] - cs_tickets["First Response Time"]
).dt.total_seconds() / 3600


# Clean the junk. 

cs_tickets_clean = cs_tickets[
    cs_tickets["resolution_hours"].notna() &
    (cs_tickets["resolution_hours"]>= 0)
].copy()

cs_tickets_clean["resolution_hours"].describe()


/usr/local/lib/python3.11/dist-packages/pandas/core/computation/expressions.py:73: RuntimeWarning: invalid value encountered in greater_equal
  return op(a, b)


count    1404.000000
mean        7.577932
std         5.596637
min         0.000000
25%         3.000000
50%         6.341667
75%        11.354167
max        23.466667
Name: resolution_hours, dtype: float64

## 3.2 Define SLA Risk

In [15]:
def sla_bucket(hours):
    if hours <= 24:
        return "low_risk"
    elif hours <= 48:
        return "medium_risk"
    else:
        return "high_risk"

cs_tickets_clean["sla_risk"] = cs_tickets_clean["resolution_hours"].apply(sla_bucket)

cs_tickets_clean["sla_risk"].value_counts(normalize=True)

sla_risk
low_risk    1.0
Name: proportion, dtype: float64

# Step 4. Creeate the KPI Table

In [16]:
kpi_table = pd.DataFrame({
    "metric": [
        "total_tickets",
        "median_resolution_hours",
        "p90_resolution_hours",
        "high_risk_ticket_pct"
    ],
    "value": [
        len(cs_tickets_clean),
        cs_tickets_clean["resolution_hours"].median(),
        cs_tickets_clean["resolution_hours"].quantile(0.9),
        (cs_tickets_clean["sla_risk"] == "high_risk").mean()
    ]
})

kpi_table

,metric,value
0,total_tickets,1404.000000
1,median_resolution_hours,6.341667
2,p90_resolution_hours,16.000000
3,high_risk_ticket_pct,0.000000


# Step 5. SLA Risk by Priority

In [17]:
priority_sla = (
    cs_tickets_clean
    .groupby("Ticket Priority")
    .agg(
        tickets=("Ticket ID", "count"),
        median_resolution_hours=("resolution_hours", "median"),
        high_risk_pct=("sla_risk", lambda x: (x == "high_risk").mean())
    )
    .sort_values("high_risk_pct", ascending=False)
)

priority_sla

,tickets,median_resolution_hours,high_risk_pct
Ticket Priority,,,
critical,374,5.983333,0.0
high,355,7.116667,0.0
low,334,7.083333,0.0
medium,341,5.866667,0.0


# Step 6. Type & Channel

Now I'll apply this same methodology to the Ticket Channel and Ticket Type fields for more comprehensive metrics. 

In [18]:
def sla_breakdown(col):
    return(
        cs_tickets_clean
        .groupby(col)
        .agg(
            tickets=("Ticket ID", "count"),
            median_resoultion_hours=("resolution_hours", "median"),
            high_risk_pct=("sla_risk", lambda x: (x == "high_risk").mean())
        )
        .sort_values("high_risk_pct", ascending=False)
    )

category_sla = sla_breakdown("Ticket Type")
channel_sla = sla_breakdown("Ticket Channel")

category_sla, channel_sla

(                      tickets  median_resoultion_hours  high_risk_pct
 Ticket Type                                                          
 billing inquiry           273                 5.266667            0.0
 cancellation request      265                 6.366667            0.0
 product inquiry           257                 6.983333            0.0
 refund request            304                 6.708333            0.0
 technical issue           305                 6.383333            0.0,
                 tickets  median_resoultion_hours  high_risk_pct
 Ticket Channel                                                 
 chat                355                 6.516667            0.0
 email               374                 6.408333            0.0
 phone               327                 6.050000            0.0
 social media        348                 6.425000            0.0)